In [41]:
from lxml import etree
import wikitextparser as wtp
import json
import re

In [43]:
WIKI_PATH = "cswiki-latest-pages-articles.xml"
CZ_DEV = "npfl140/data/CZ.dev.jsonl"
TXT_FOLDER_PATH = "npfl140/data/txt_extracted"

In [3]:
tag_set = set()
context = etree.iterparse(WIKI_PATH, events=("start",))

for _, elem in context:
    tag_set.add(elem.tag)
    if len(tag_set) >= 100:
        break

print("Top-level tags found:")
for tag in tag_set:
    print("-", tag)

Top-level tags found:
- {http://www.mediawiki.org/xml/export-0.11/}ns
- {http://www.mediawiki.org/xml/export-0.11/}timestamp
- {http://www.mediawiki.org/xml/export-0.11/}namespaces
- {http://www.mediawiki.org/xml/export-0.11/}mediawiki
- {http://www.mediawiki.org/xml/export-0.11/}sitename
- {http://www.mediawiki.org/xml/export-0.11/}siteinfo
- {http://www.mediawiki.org/xml/export-0.11/}contributor
- {http://www.mediawiki.org/xml/export-0.11/}revision
- {http://www.mediawiki.org/xml/export-0.11/}sha1
- {http://www.mediawiki.org/xml/export-0.11/}comment
- {http://www.mediawiki.org/xml/export-0.11/}namespace
- {http://www.mediawiki.org/xml/export-0.11/}format
- {http://www.mediawiki.org/xml/export-0.11/}username
- {http://www.mediawiki.org/xml/export-0.11/}minor
- {http://www.mediawiki.org/xml/export-0.11/}id
- {http://www.mediawiki.org/xml/export-0.11/}ip
- {http://www.mediawiki.org/xml/export-0.11/}text
- {http://www.mediawiki.org/xml/export-0.11/}parentid
- {http://www.mediawiki.org/xm

In [9]:
def parse_wiki_dump(xml_file, max_pages=10):
    context = etree.iterparse(xml_file, events=('end',), tag='{*}page')
    for _, elem in context:
        title = elem.findtext('{*}title')
        text_elem = elem.find('.//{*}revision/{*}text')
        text = text_elem.text if text_elem is not None else ''

        print(f"Title: {title}")
        print(f"Content: {text[:1000]}...")
        print("=" * 80)

        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]

        max_pages -= 1
        if max_pages == 0:
            break

In [10]:
parse_wiki_dump(WIKI_PATH)

Title: Hlavní strana
Content: <templatestyles src="MediaWiki:MainPage-stylesheet.css" />{{nobots}}__NOTOC__ __NOEDITSECTION__
<!------- HLAVIČKA ------->
<div class="header nomobile">
<div class="header-left">
<div class="header-welcome-headline">[[Wikipedie:Průvodce|Vítejte ve&nbsp;Wikipedii]],</div>
<div class="header-welcome-description">internetové [[encyklopedie|encyklopedii]], kterou může [[Nápověda:Úvod|upravovat každý]].<br>[[Česká Wikipedie]] má nyní '''[[Speciální:Statistika|{{POČETČLÁNKŮ}}]]''' [[Speciální:Nové stránky|článků]].</div>
</div>
<div class="header-right">[[Soubor:Other languages icon.svg|80px|text-bottom|link=]]<br>[[Wikipedie:Seznam jazyků Wikipedie|Jiné jazyky]]<br>[[Wikipedie:Velvyslanectví|Embassy]]</div>
</div>
<!------- PANORAMATICKÝ OBRÁZEK -------><!--
<div style="background-color:#FCFCFC; border:1px solid #E6E6E6; margin-top:.5em">
<div style="background-color:#E6E6E6; padding-top:.2em; padding-left:.4em; padding-bottom:.2em">'''[[Wikipedie:Obrázek týdn

In [16]:
def parse_and_clean(xml_path, max_pages=None, output_json=None):
    context = etree.iterparse(xml_path, events=('end',), tag='{*}page')
    articles = []

    for _, elem in context:
        title = elem.findtext('{*}title')
        text_elem = elem.find('.//{*}revision/{*}text')
        raw_text = text_elem.text if text_elem is not None else ""

        parsed = wtp.parse(raw_text)

        clean_text = parsed.plain_text()

        if not clean_text.strip() or title.startswith(("Wikipedie:", "Nápověda:", "Soubor:", "Šablona:")):
            continue

        articles.append({
            "title": title,
            "clean_text": clean_text.strip()
        })

        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]

        if max_pages and len(articles) >= max_pages:
            break


    if output_json:
        with open(output_json, "w", encoding="utf-8") as f:
            json.dump(articles, f, ensure_ascii=False, indent=2)

    return articles

In [19]:
results = parse_and_clean(WIKI_PATH, max_pages=50)

for article in results[:3]:
    print(f"Title: {article['title']}")
    print(article['clean_text'])
    print("=" * 80)

Title: Hlavní strana
__NOTOC__ __NOEDITSECTION__



Vítejte ve Wikipedii,
internetové encyklopedii, kterou může upravovat každý.Česká Wikipedie má nyní  článků.

Jiné jazykyEmbassy






Článek týdne

Nejlepší články • Dobré články • Další články týdne…



Víte, že…





Listování kategoriemi

Všechny kategorie…



Tematické portály

Dobré portály • Všechny portály…



Wikipedie

Další nápověda…



Ostatní projekty

Další informace…






Obrázek týdne

Nejlepší obrázky • Další obrázky týdne…



Aktuality

Další aktuality…



.  v minulosti

Další výročí…
Title: Astronomie
Astronomie, řecky αστρονομία z άστρον (astron) hvězda a νόμος (nomos) zákon, česky též hvězdářství, je věda, která se zabývá jevy za hranicemi zemské atmosféry. Zvláště tedy výzkumem vesmírných těles, jejich soustav, různých dějů ve vesmíru i vesmírem jako celkem.

== Historie astronomie ==

=== Antika ===
Astronomie se podobně jako další vědy začala rozvíjet ve starověku. Na území Babylonie však nebylo k popisu použ

In [20]:
def get_page_by_title(xml_path, search_title):
    context = etree.iterparse(xml_path, events=('end',), tag='{*}page')
    for _, elem in context:
        title = elem.findtext('{*}title')
        if title == search_title:
            text_elem = elem.find('.//{*}revision/{*}text')
            raw_text = text_elem.text if text_elem is not None else ""
            return title, raw_text.strip()
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    return None, None

In [25]:
title, content = get_page_by_title(WIKI_PATH, "Karel Plíhal")
if content:
    print(f"Title: {title}\n\n{content}...")
else:
    print("Page not found.")

Title: Karel Plíhal

{{Infobox - hudební umělec
  | jméno = Karel Plíhal
  | barva pozadí = sólový zpěvák
  | původ = {{Vlajka a název|Česko}}
  | nástroje = 
  | žánr = [[folk]]<br />[[jazz]]
  | povolání = zpěvák, kytarista, skladatel
  | ocenění = [[Ceny Anděl 2012]]<br />4× cena festivalu [[Porta]]
  | aktivní roky = od [[1983]]<ref>https://karelplihal.cz/2023/01/11/karel-plihal-nikdy-jsem-se-ze-sebe-neposral/</ref> (samostatně)
  | vydavatel = 
  | příbuzná témata = 
  | web = [http://www.karelplihal.cz www.karelplihal.cz]
  | významný nástroj = [[kytara]]
}}
'''Karel Plíhal''' (* [[23. srpen|23. srpna]] [[1958]] [[Přerov]]) je [[Česko|český]] kytarista, zpěvák, skladatel, textař, básník, hudební režisér a aranžér.

Vystudoval [[Střední průmyslová škola strojnická Olomouc|Střední průmyslovou školu strojnickou]] v [[Olomouc]]i, poté pracoval jako konstruktér, topič v olomouckém divadle a nakonec se stal písničkářem, ačkoli nemá hudební vzdělání. Na [[kytara|kytaru]] hraje od patnác

In [27]:
def clean_wikipedia_article(raw_text: str) -> str:
    # Step 1: Parse using wikitextparser
    parsed = wtp.parse(raw_text)

    # Step 2: Remove templates (like infoboxes, navboxes)
    for template in parsed.templates:
        raw_text = raw_text.replace(str(template), '')

    # Step 3: Remove reference tags <ref>...</ref> and <references />
    raw_text = re.sub(r'<ref[^>]*>.*?</ref>', '', raw_text, flags=re.DOTALL)
    raw_text = re.sub(r'<references\s*/?>', '', raw_text)

    # Step 4: Remove categories, DEFAULTSORT, authority control, etc.
    raw_text = re.sub(r'\{\{.*?\}\}', '', raw_text, flags=re.DOTALL)
    raw_text = re.sub(r'\[\[Kategorie:[^\]]+\]\]', '', raw_text)
    raw_text = re.sub(r'\{\{DEFAULTSORT:[^\}]+\}\}', '', raw_text)

    # Step 5: Clean up wiki links: [[link|text]] → text, [[text]] → text
    raw_text = re.sub(r'\[\[([^|\]]*\|)?([^\]]+)\]\]', r'\2', raw_text)

    # Step 6: Remove file/image links (e.g. [[Soubor:X.jpg|text]])
    raw_text = re.sub(r'\[\[Soubor:[^\]]+\]\]', '', raw_text)
    raw_text = re.sub(r'\[\[File:[^\]]+\]\]', '', raw_text)

    # Step 7: Remove external links [http... text] → text
    raw_text = re.sub(r'\[http[^\s]+\s([^\]]+)\]', r'\1', raw_text)
    raw_text = re.sub(r'\[http[^\]]+\]', '', raw_text)

    # Step 8: Remove extra braces, comments, and formatting artifacts
    raw_text = re.sub(r'<!--.*?-->', '', raw_text, flags=re.DOTALL)
    raw_text = re.sub(r"''+", '', raw_text)  # remove ''italic'' or '''bold'''

    # Step 9: Remove headings like == Heading ==
    raw_text = re.sub(r'^=+\s*(.*?)\s*=+$', r'\1', raw_text, flags=re.MULTILINE)

    # Step 10: Normalize whitespace
    cleaned = re.sub(r'\n{2,}', '\n\n', raw_text)
    cleaned = re.sub(r'[ \t]+', ' ', cleaned)
    cleaned = cleaned.strip()

    return cleaned

In [29]:
title, content = get_page_by_title(WIKI_PATH, "PSJ")
if content:
    cleaned_text = clean_wikipedia_article(content)
    print(f"Title: {title}\n\n{cleaned_text}...")
else:
    print("Page not found.")

Title: PSJ

PSJ, a.s. je bývalá česká stavební společnost. Vznikla v roce 1990; postupně vytvořila velkou společnost s holdingovým uskupením firem a soustřeďovala kolem 1400 zaměstnanců a měla miliardové obraty. Následkem nepovedených zakázek v Rusku i v Česku je však společnost od roku 2018 v insolvenci a v podstatě žádnou činnost již nevykonává.

Struktura skupiny
Holdingové uskupení PSJ pokrývalo široké spektrum pozemního stavitelství, developmentu, exportních aktivit, vybraných stavebních řemesel a dodávek technologií. Nabídku činností firem pod hlavičkou PSJ tvořily vodohospodářské a inženýrské stavby (D.I.S., spol. s r.o.), dopravní infrastruktura (ALPINE Bau CZ, a.s.), obalovny a asfaltové technologie (SILASFALT s.r.o.), technologické a energetické stavby (PSJ Hydrotranzit, a.s.), technika prostředí a zdravotechnické instalace (EKOKLIMA, a.s.), elektroinstalace (Inovat SE) a facility management (SCF Servis, s.r.o.). Společnost PSJ je dlouholetým majoritním vlastníkem a generální

In [45]:
def sanitize_filename(title):
    return "".join(c if c.isalnum() or c in (' ', '.', '_', '-') else '_' for c in title).strip().replace(' ', '_')

with open(CZ_DEV, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        title_json = data['wikititle'].replace("_", " ")
        print(title_json)
        title, content = get_page_by_title(WIKI_PATH, title_json)
        if not title or not content:
            continue

        cleaned_text = clean_wikipedia_article(content)
        filename = TXT_FOLDER_PATH + "/" + sanitize_filename(title) + ".txt"

        with open(filename, 'w', encoding='utf-8') as txtfile:
            txtfile.write(f"{title}\n\n{cleaned_text}")

        print(f"Saved: {filename}")

Karel Plíhal
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Karel_Plíhal.txt
Jana Petrů
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Jana_Petrů.txt
Čechoameričané
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Čechoameričané.txt
Svatý Mikuláš
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Svatý_Mikuláš.txt
Vodní nádrž Slapy
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Vodní_nádrž_Slapy.txt
Disident
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Disident.txt
Jan Neruda
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Jan_Neruda.txt
Prachovské skály
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Prachovské_skály.txt
Zlatý slavík
Saved: /Users/vladyslav.furda/PycharmProjects/npfl140/data/txt_extracted/Zlatý_slavík.txt
Oldřich Kaiser
Saved: /Users/vladyslav.furda/PycharmProjects/npfl

Check number of pages

In [46]:
def count_pages(xml_path: str) -> int:
    count = 0
    context = etree.iterparse(xml_path, events=("end",), tag="{*}page")
    for _, elem in context:
        count += 1
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    return count

In [47]:
total_pages = count_pages(WIKI_PATH)

In [54]:
print("Count of pages:", format(total_pages, ","))

Count of pages: 1,216,713


In [55]:
from lxml import etree

def count_main_articles(xml_path: str) -> int:
    count = 0
    context = etree.iterparse(xml_path, events=("end",), tag="{*}page")
    for _, elem in context:
        ns = elem.findtext('{*}ns')
        if ns == "0":  # only count main namespace articles
            count += 1
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    return count

In [57]:
main_articles = count_main_articles(WIKI_PATH)

In [59]:
print("Count of pages:", format(main_articles, ","))

Count of pages: 926,274
